<a href="https://colab.research.google.com/github/burjoawl/Burrs_Portofolio/blob/main/The%20Look/The_Look.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

## Menyatukan Colab dengan Cloud

Proses untuk menyatukan Google Colab dengan Google Cloud

In [ ]:
from google.colab import auth    # import autentikasi dari google colab
auth.authenticate_user()         # autentikasi user google
print('Authenticated')           # jika berhasil akan muncul kalimat 'Authenticated'

from google.cloud import bigquery             # import bigquery dari google cloud
project_id = "hacktiv8-farhan-1"              # nama project id dari google cloud, pada bagian 'select a project', kolom 'ID'
client = bigquery.Client(project=project_id)  # membuat variabel client dari bigquery agar mempersingkat

Authenticated


## Google Cloud

Menggunakan data dari
[BigQuery](https://console.cloud.google.com/bigquery?ws=!1m4!1m3!3m2!1sbigquery-public-data!2sthelook_ecommerce) untuk melakukan pengerjaan



In [ ]:
# Membuat variabel BigQuery yang berisi tabel orders dari thelook_ecommerce
BigQuery = client.query ('''
SELECT *
FROM bigquery-public-data.thelook_ecommerce.orders
''').to_dataframe()

# Melihat tipe data BigQuery
BigQuery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125303 entries, 0 to 125302
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   order_id      125303 non-null  Int64              
 1   user_id       125303 non-null  Int64              
 2   status        125303 non-null  object             
 3   gender        125303 non-null  object             
 4   created_at    125303 non-null  datetime64[ns, UTC]
 5   returned_at   12296 non-null   datetime64[ns, UTC]
 6   shipped_at    81131 non-null   datetime64[ns, UTC]
 7   delivered_at  43519 non-null   datetime64[ns, UTC]
 8   num_of_item   125303 non-null  Int64              
dtypes: Int64(3), datetime64[ns, UTC](4), object(2)
memory usage: 9.0+ MB


# Problem Statement

Bagaimana cara untuk menaikkan penjualan sebesar 10% pada The Look E-Commerce saat Q4 Tahun 2022?


Smart Analysis:
1. Specific     = Menaikkan penjualan produk-produk pada kuartil selanjutnya
2. Measurable   = Naiknya penjualan sebesar 10% dari Kuartil sebelumnya
3. Achievable   = Perlu dianalisis dari data penjualan pada kuartil sebelumnya
4. Relevant     = The Look E-Commerce ingin menaikkan penjualan dibandingkan sebelum-sebelumnya
5. Time-Bound   = Meningkatnya penjualan pada Kuartil Keempat di Tahun 2022

# Query

## Penjabaran 1

Berdasarkan tabel tersebut dapat dilihat bahwa bulan kedua mengalami penurunan transaksi tetapi setelah itu naik sampai bulan kesembilan, transaksi terbanyak terjadi pada bulan kesembilan dan paling sedikit terjadi pada bulan kedua.




In [ ]:
df_1 = client.query ('''                                                                    # Membuat variabel df_1 untuk Penjabaran 1
SELECT 	                                                                                    # Query kedua untuk membuat selisih nilai 'Transaksi'
  Kuartil,
  Bulan,
  Tahun,
  Transaksi,
  LAG(Transaksi) OVER (ORDER BY Bulan) AS `Transaksi Sebelumnya`,                           # Membuat kolom baru yang berisi nilai pada kolom 'Transaksi' sebelumnya dengan fungsi Lag dan diurut berdasarkan bulan
  Transaksi - LAG(Transaksi) OVER (ORDER BY Bulan) AS `Selisih Transaksi Sebelumnya`        # Membuat kolom baru yang berisi nilai selisih antara 'Transaksi' dengan 'Transaksi Sebelumnya' dan diurut berdasarkan bulan
FROM (
        SELECT                                                                              # Query pertama untuk mengambil kolom yang ingin dipilih
                EXTRACT (quarter from created_at) AS Kuartil,                               # Mengambil quarter dari 'kolom created_at' dan menamai sebagai 'Kuartil'
                EXTRACT (month from created_at) AS Bulan,                                   # Mengambil month dari 'kolom created_at' dan menamai sebagai 'Bulan'
                EXTRACT (year from created_at) AS Tahun,                                    # Mengambil year dari 'kolom created_at' dan menamai sebagai 'Tahun'
                COUNT (order_id) AS Transaksi                                               # Menghitung jumlah 'order_id' dan menamai sebagai Transaksi
        FROM
                bigquery-public-data.thelook_ecommerce.orders                               # Mengambil tabel dari orders
        WHERE
                status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"    # Menyaring data dengan status 'Complete' dan pada tahun 2022
        GROUP BY
                Kuartil, Bulan, Tahun                                                       # Di grup berdasarkan Tahun, Bulan, dan Kuartil
        HAVING
                Kuartil < 4                                                                 # Karena Kuartil baru dibuat pada baris SELECT, jadi menggunakan HAVING
        ORDER BY
                Kuartil, Bulan, Tahun ASC                                                   # Di urutkan berdasarkan angka dari kecil sampai terbesar
      )
ORDER BY
        Kuartil, Bulan, Tahun ASC
''').to_dataframe()

df_1                                                                                        # Menampilkan variabel df_1 yang sudah dibuat


,Kuartil,Bulan,Tahun,Transaksi,Transaksi Sebelumnya,Selisih Transaksi Sebelumnya
0,1,1,2022,707,<NA>,<NA>
1,1,2,2022,605,707,-102
2,1,3,2022,751,605,146
3,2,4,2022,765,751,14
4,2,5,2022,774,765,9
5,2,6,2022,807,774,33
6,3,7,2022,880,807,73
7,3,8,2022,968,880,88
8,3,9,2022,1021,968,53


## Penjabaran 2

Berdasarkan tabel tersebut dapat dilihat bahwa bulan kedua dan bulan keempat mengalami penurunan penjualan tetapi selain dua bulan tersebut mengalami kenaikan yang dapat menutupi selisih dari dua bulan tersebut, penjualan terbesar terjadi pada bulan kesembilan dan paling sedikit terjadi pada bulan kedua.

In [ ]:
df_2 = client.query ('''                                                                          # Membuat variabel df_2 untuk Penjabaran 2
SELECT 	                                                                                          # Query kedua untuk membuat selisih nilai 'Penjualan'
  Kuartil,
  Bulan,
  Tahun,
  `Penjualan USD`,
  LAG(`Penjualan USD`) OVER (ORDER BY Bulan) AS `Penjualan Sebelumnya`,                           # Membuat kolom baru yang berisi nilai pada kolom 'Penjualan USD' sebelumnya dengan fungsi Lag dan diurut berdasarkan bulan
  `Penjualan USD` - LAG(`Penjualan USD`) OVER (ORDER BY Bulan) AS `Selisih Penjualan Sebelumnya`  # Membuat kolom baru yang berisi nilai selisih antara 'Penjualan USD' dengan 'Penjualan Sebelumnya' dan diurut berdasarkan bulan
FROM (
        SELECT                                                                                    # Query pertama untuk mengambil kolom yang ingin dipilih
                EXTRACT (quarter from created_at) AS Kuartil,                                     # Mengambil quarter dari 'kolom created_at' dan menamai sebagai 'Kuartil'
                EXTRACT (month from created_at) AS Bulan,                                         # Mengambil month dari 'kolom created_at' dan menamai sebagai 'Bulan'
                EXTRACT (year from created_at) AS Tahun,                                          # Mengambil year dari 'kolom created_at' dan menamai sebagai 'Tahun'
                SUM (sale_price) AS `Penjualan USD`                                               # Menghitung total 'sale_price' dan menamai sebagai Penjualan USD
        FROM
                bigquery-public-data.thelook_ecommerce.order_items                                # Mengambil tabel dari orders
        WHERE
                status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"          # Menyaring data dengan status 'Complete' dan pada tahun 2022
        GROUP BY
                Kuartil, Bulan, Tahun                                                             # Di grup berdasarkan Tahun, Bulan, dan Kuartil
        HAVING
                Kuartil < 4                                                                       # Karena Kuartil baru dibuat pada baris SELECT, jadi menggunakan HAVING
        ORDER BY
                Kuartil, Bulan, Tahun ASC                                                         # Di urutkan berdasarkan angka dari kecil sampai terbesar
      )
ORDER BY
        Kuartil, Bulan, Tahun ASC
''').to_dataframe()

df_2                                                                                              # Menampilkan variabel df_2 yang sudah dibuat

,Kuartil,Bulan,Tahun,Penjualan USD,Penjualan Sebelumnya,Selisih Penjualan Sebelumnya
0,1,1,2022,65183.190080,NaN,NaN
1,1,2,2022,59067.140080,65183.190080,-6116.050000
2,1,3,2022,64841.030030,59067.140080,5773.889950
3,2,4,2022,71080.520084,64841.030030,6239.490055
4,2,5,2022,71339.730093,71080.520084,259.210009
5,2,6,2022,66357.870065,71339.730093,-4981.860028
6,3,7,2022,72088.330051,66357.870065,5730.459986
7,3,8,2022,83890.500099,72088.330051,11802.170048
8,3,9,2022,90007.200077,83890.500099,6116.699978


## Penjabaran 3

Berdasarkan tabel tersebut dapat dilihat bahwa bulan kedua mengalami penurunan orang yang bertransaksi tetapi selain bulan tersebut mengalami kenaikan hingga bulan kesembilan, orang yang bertransaksi terbesar terjadi pada bulan kesembilan dan paling sedikit terjadi pada bulan kedua.

In [ ]:
df_3 = client.query ('''                                                                                 # Membuat variabel df_3 untuk Penjabaran 3
 SELECT 	                                                                                               # Query kedua untuk membuat selisih nilai 'Transaksi'
  Kuartil,
  Bulan,
  Tahun,
  `Transaksi User`,
  LAG(`Transaksi User`) OVER (ORDER BY Bulan) AS `Transaksi User Sebelumnya`,                            # Membuat kolom baru yang berisi nilai pada kolom 'Penjualan USD' sebelumnya dengan fungsi Lag dan diurut berdasarkan bulan
  `Transaksi User` - LAG(`Transaksi User`) OVER (ORDER BY Bulan) AS `Selisih Transaksi User Sebelumnya`  # Membuat kolom baru yang berisi nilai selisih antara 'Penjualan USD' dengan 'Penjualan Sebelumnya' dan diurut berdasarkan bulan

From (
          SELECT                                                                                         # Query pertama untuk mengambil kolom yang ingin dipilih
                  EXTRACT (quarter from created_at) AS Kuartil,                                          # Mengambil quarter dari 'kolom created_at' dan menamai sebagai 'Kuartil'
                  EXTRACT (month from created_at) AS Bulan,                                              # Mengambil month dari 'kolom created_at' dan menamai sebagai 'Bulan'
                  EXTRACT (year from created_at) AS Tahun,                                               # Mengambil year dari 'kolom created_at' dan menamai sebagai 'Tahun'
                  COUNT (DISTINCT user_id) AS `Transaksi User`                                           # Menghitung jumlah user 'user_id', karena terdapat duplikat sehingga digunakan DISTINCT dan menamai sebagai Transaksi User
          FROM
                  bigquery-public-data.thelook_ecommerce.orders                                          # Mengambil tabel dari orders
          WHERE
                  status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"               # Menyaring data dengan status 'Complete' dan pada tahun 2022
          GROUP BY
                  Kuartil, Bulan, Tahun                                                                  # Di grup berdasarkan Tahun, Bulan, dan Kuartil
          HAVING
                  Kuartil < 4                                                                            # Karena Kuartil baru dibuat pada baris SELECT, jadi menggunakan HAVING
          ORDER BY
                  Kuartil, Bulan, Tahun ASC                                                              # Di urutkan berdasarkan angka dari kecil sampai terbesar
    )
ORDER BY
                  Kuartil, Bulan, Tahun ASC
''').to_dataframe()

df_3                                                                                                     # Menampilkan variabel df_3 yang sudah dibuat

,Kuartil,Bulan,Tahun,Transaksi User,Transaksi User Sebelumnya,Selisih Transaksi User Sebelumnya
0,1,1,2022,703,<NA>,<NA>
1,1,2,2022,602,703,-101
2,1,3,2022,748,602,146
3,2,4,2022,761,748,13
4,2,5,2022,773,761,12
5,2,6,2022,802,773,29
6,3,7,2022,874,802,72
7,3,8,2022,961,874,87
8,3,9,2022,1018,961,57


## Penjabaran 4

Berdasarkan tabel tersebut dapat dilihat bahwa kategori terbanyak adalah kategori `Intimates` yang terdapat pada bulan ketiga sampai bulan kesembilan, pada bulan pertama kategori terbanyak yaitu `Fashion Hoodies & Sweatshirts	` dan bulan kedua yaitu `Jeans`, dan transaksi terbanyak terjadi pada bulan kesembilan.

In [ ]:
df_4 = client.query ('''
SELECT *                                                                                                 # Query ketiga hanya untuk memilih Urutan_Transaksi nomor 1
FROM (
        SELECT                                                                                           # Query kedua ini untuk mengambil rank 1 pada tiap bulan
                ROW_NUMBER()                                                                             # Row Number untuk membuat ranking berdasarkan Ranking_Transaksi, dan sebagai window function
                OVER (                                                                                   # Over berguna untuk membuat urutan berdasarkan yang didalam kurung
                  PARTITION BY Kuartil,Bulan,Tahun                                                       # Partition berguna untuk membagi baris berdasarkan Kuartil, Bulan, dan Tahun, jadi untuk membuat hitungan rank terbagi dari baris tersebut
                  ORDER BY Jumlah_Transaksi DESC) AS Ranking_Transaksi,                                  # Order By berguna untuk membuat urutan berdasarkan Jumlah_Transaksi secara dari besar ke kecil yang berguna untuk mendapatkan nilai Ranking_Transaksi tertinggi
                Kuartil, Bulan, Tahun, Kategori, Jumlah_Transaksi                                        # dan dinamai sebagai Ranking_Transaksi
        FROM  (
                      SELECT                                                                             # Query pertama ini untuk menggabungkan antara 2 tabel, order_items dan products agar menyatukan kolom 'category' dan 'status'
                           EXTRACT (quarter from created_at) AS Kuartil,                                 # Hasil yang didapatkan berupa semua kategori yang ada dalam bulan yang sama beserta jumlah transaksinya
                           EXTRACT (month from created_at) AS Bulan,                                     # Extract sama seperti sebelum-sebelumnya
                           EXTRACT (year from created_at) AS Tahun,
                           category AS Kategori,                                                         # Mengganti nama dari category menjadi Kategori
                           COUNT(sale_price) as Jumlah_Transaksi                                         # COUNT untuk menghitung berapa banyak transaksi dari sale_price
                      FROM
                            bigquery-public-data.thelook_ecommerce.order_items AS Order_Items            # Mengambil tabel order_items dan menamai sebagai Order_Items
                      JOIN
                           bigquery-public-data.thelook_ecommerce.products AS Products                   # Mengambil tabel products dan menamai sebagai Products
                      ON
                           Order_Items.product_id = Products.id                                          # Menggabungkan tabel Order_Items dengan Products, Order_Items dengan Products, dan product_id dengan id
                      WHERE
                           status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"
                      GROUP BY
                            Kuartil,Bulan,Tahun,Kategori
                      HAVING Kuartil < 4
                 ) AS Transaksi                                                                          # Query pertama dinamai sebagai Transaksi
    ) AS Rank_Transaksi                                                                                  # Query ketiga dinamai sebagai Rank_Transaksi
WHERE Ranking_Transaksi = 1
ORDER BY Tahun,Bulan
''').to_dataframe()

df_4

,Ranking_Transaksi,Kuartil,Bulan,Tahun,Kategori,Jumlah_Transaksi
0,1,1,1,2022,Intimates,84
1,1,1,2,2022,Sweaters,73
2,1,1,3,2022,Intimates,77
3,1,2,4,2022,Intimates,91
4,1,2,5,2022,Fashion Hoodies & Sweatshirts,91
5,1,2,6,2022,Intimates,83
6,1,3,7,2022,Jeans,97
7,1,3,8,2022,Fashion Hoodies & Sweatshirts,105
8,1,3,9,2022,Jeans,120


## Penjabaran 5

Berdasarkan tabel tersebut dapat dilihat bahwa kategori terbanyak dalam segi pendapatan adalah adalah kategori `Outerwear & Coals` yang terdapat pada semua bulan, tetapi bulan kedua keenam berbeda yaitu kategori `Jeans`, dan pendapatan terbesar terjadi pada bulan bulan kesembilan.

In [ ]:
df_5 = client.query ('''
SELECT *                                                                                                 # Query ketiga hanya untuk memilih Urutan_Transaksi nomor 1
FROM (
        SELECT                                                                                           # Query kedua ini untuk mengambil rank 1 pada tiap bulan
                ROW_NUMBER()                                                                             # Row Number untuk membuat ranking berdasarkan
                OVER (                                                                                   # Over berguna untuk membuat urutan berdasarkan yang didalam kurung
                  PARTITION BY Kuartil,Bulan,Tahun                                                       # Partition berguna untuk membagi baris berdasarkan Kuartil, Bulan, dan Tahun
                  ORDER BY Total_Pendapatan DESC) AS Urutan_Transaksi,                                   # Order By berguna untuk membuat urutan berdasarkan Total_Pendapatan secara dari besar ke kecil yang berguna untuk mendapatkan nilai Total_Pendapatan tertinggi
                Kuartil, Bulan, Tahun, Kategori, Total_Pendapatan                                        # dan dinamai sebagai Urutan_Transaksi
        FROM  (
                      SELECT                                                                             # Query pertama ini untuk menggabungkan antara 2 tabel, order_items dan products agar menyatukan kolom 'category' dan 'status'
                           EXTRACT (quarter from created_at) AS Kuartil,                                 # Hasil yang didapatkan berupa semua kategori yang ada dalam bulan yang sama beserta jumlah transaksinya
                           EXTRACT (month from created_at) AS Bulan,
                           EXTRACT (year from created_at) AS Tahun,
                           category AS Kategori,
                           SUM(sale_price) as Total_Pendapatan                                           # SUM untuk menghitung total pendapatan dari sale_price
                      FROM
                            bigquery-public-data.thelook_ecommerce.order_items AS Order_Items            # Mengambil tabel order_items dan menamai sebagai Order_Items
                      JOIN
                           bigquery-public-data.thelook_ecommerce.products AS Products                   # Mengambil tabel products dan menamai sebagai Products
                      ON
                           Order_Items.product_id = Products.id                                          # Menggabungkan tabel Order_Items dengan Products, Order_Items dengan Products, dan product_id dengan id
                      WHERE
                           status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"
                      GROUP BY
                            Kuartil,Bulan,Tahun,Kategori
                      HAVING Kuartil < 4
                 ) AS Transaksi                                                                          # Query pertama dinamai sebagai Transaksi
    ) AS Rank_Transaksi                                                                                  # Query ketiga dinamai sebagai Rank_Transaksi
WHERE Urutan_Transaksi = 1
ORDER BY Tahun,Bulan
''').to_dataframe()

df_5

,Urutan_Transaksi,Kuartil,Bulan,Tahun,Kategori,Total_Pendapatan
0,1,1,1,2022,Outerwear & Coats,7731.309965
1,1,1,2,2022,Jeans,7779.730017
2,1,1,3,2022,Outerwear & Coats,9572.149988
3,1,2,4,2022,Outerwear & Coats,10890.789965
4,1,2,5,2022,Outerwear & Coats,9445.239994
5,1,2,6,2022,Outerwear & Coats,8559.630026
6,1,3,7,2022,Jeans,8848.020023
7,1,3,8,2022,Outerwear & Coats,11327.749989
8,1,3,9,2022,Jeans,13712.430012


## Penjabaran 6

Berdasarkan tabel tersebut dapat dilihat bahwa korelasi antara `Transaksi User` dengan `Jumlah Transaksi` lebih berkaitan dibandingkan korelasi antara `Transaksi User` dengan `Penjualan` dimana barang dengan jumlah transaksi tertinggi belum tentu sebanding dengan penjualan.

In [ ]:
df_6 = client.query ('''
SELECT CORR(`Transaksi User`, `Jumlah Transaksi`) AS `Transaksi User dengan Jumlah Transaksi`, # Membuat korelasi antara Transaksi User (Sumbu X) dengan Jumlah Transaksi (Sumbu Y) dengan fungsi Corr
       CORR(`Transaksi User`, `Penjualan USD`) AS `Transaksi User dengan Penjualan`            # Membuat korelasi antara Transaksi User (Sumbu X) dengan Penjualan USD (Sumbu Y) dengan fungsi Corr

FROM (
        SELECT                                                                                 # Membuat query yang mengambil jumlah tetapi dibagi berdasarkan kategori
                EXTRACT (quarter from created_at) AS Kuartil,
                EXTRACT (month from created_at) AS Bulan,
                EXTRACT (year from created_at) AS Tahun,
                category as Kategori,
                COUNT (order_id) AS `Jumlah Transaksi`,
                SUM (sale_price) AS `Penjualan USD`,
                COUNT (DISTINCT user_id) AS `Transaksi User`
        FROM
                bigquery-public-data.thelook_ecommerce.order_items AS Order_Items             # Mengambil tabel order_items dan menamai sebagai Order_Items
        JOIN
                bigquery-public-data.thelook_ecommerce.products AS Products                   # Mengambil tabel products dan menamai sebagai Products
        ON
                Order_Items.product_id = Products.id
        WHERE
                status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"
        GROUP BY
                Kuartil, Bulan, Tahun, Kategori
        HAVING
                Kuartil < 4
        )
''').to_dataframe()

df_6

,Transaksi User dengan Jumlah Transaksi,Transaksi User dengan Penjualan
0,0.998774,0.70724


## Kesimpulan

Berdasarkan penjabaran-penjabaran diatas, dapat disimpulkan bahwa
- Pada kuartal 1 jumlah transaksi dan pendapatan lebih sedikit dibandingkan kuartal 3, dimana kuartal 3 memiliki jumlah yang lumayan banyak dibandingkan kuartal 1 dan 2
- Berdasarkan analisis antara transaksi dengan penjualan, dapat diketahui bahwa barang yang paling laku terjual belum tentu merupakan barang yang paling banyak menghasilkan pendapatan, sebagai contoh :
  - Untuk kategori terbanyak yang terjual adalah `Intimates` pada tiap bulan dari bulan ketiga sampai kesembilan, untuk bulan pertama kategori terbanyak yaitu `Fashion Hoodies & Sweatshirts	` dan bulan kedua yaitu `Jeans`
  - Meskipun kategori `Intimates` terbanyak terjual tetapi dalam segi pendapatan lebih banyak pada kategori `Outerwear & Coals`.

# Pertanyaan

## Nomer 1

Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!






**Bagaimana cara untuk menaikkan penjualan sebesar 10% pada The Look E-Commerce saat Q4 Tahun 2022?**

Sesuai dengan hasil penjabaran, terlihat bahwa pertumbuhan dari Kuartil 1 ke Kuartil 2 tumbuh sebesar 10.41% dan dari Kuartil 2 ke Kuartil 3 tumbuh sebesar 17.82%, sehingga dari Kuartil 3 ke Kuartil 4 berkemungkinan mencapai lebih dari 10%.

In [ ]:
df_2_2 = df_2.groupby(['Kuartil'])["Penjualan USD"].sum().pct_change().mul(100)
print (df_2_2)

Kuartil
1          NaN
2    10.411242
3    17.821748
Name: Penjualan USD, dtype: float64


## Nomer 2

Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

Tidak masuk akal, karena pada kuartal ketiga khususnya bulan kesembilan hanya mendapatkan `85385.310061	` dimana itu jauh dengan target awal `250000`, dan untuk laju tiap bulannya tidak sebesar itu agar mencapai target yang hanya `71354.805629`

In [ ]:
df_nomor_2 = client.query('''
 SELECT AVG (`Penjualan USD`) as `Rata-rata Pendapatan Perbulan`                                  # Query kedua untuk mengambil rata-rata dari 'Penjualan USD'
 FROM (
        SELECT                                                                                    # Query pertama untuk mengambil kolom yang ingin dipilih
                EXTRACT (quarter from created_at) AS Kuartil,                                     # Mengambil quarter dari 'kolom created_at' dan menamai sebagai 'Kuartil'
                EXTRACT (month from created_at) AS Bulan,                                         # Mengambil month dari 'kolom created_at' dan menamai sebagai 'Bulan'
                EXTRACT (year from created_at) AS Tahun,                                          # Mengambil year dari 'kolom created_at' dan menamai sebagai 'Tahun'
                SUM (sale_price) AS `Penjualan USD`                                               # Menghitung total 'sale_price' dan menamai sebagai Penjualan USD
        FROM
                bigquery-public-data.thelook_ecommerce.order_items                                # Mengambil tabel dari orders
        WHERE
                status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"          # Menyaring data dengan status 'Complete' dan pada tahun 2022
        GROUP BY
                Kuartil, Bulan, Tahun                                                             # Di grup berdasarkan Tahun, Bulan, dan Kuartil
        HAVING
                Kuartil < 4                                                                       # Karena Kuartil baru dibuat pada baris SELECT, jadi menggunakan HAVING
        ORDER BY
                Kuartil, Bulan, Tahun ASC
)
''').to_dataframe()

df_nomor_2

,Rata-rata Pendapatan Perbulan
0,71539.501184


## Nomer 3

CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

Berdasarkan pertumbuhan user, transaksi, dan GMV, terlihat adanya trend yang naik dari Kuartil 1 hingga Kuartil 3 pada tahun 2022
- User naik 13.25% pada Kuartil 2 dan 22.79% pada Kuartil 3
- Transaksi naik 10.54% pada Kuartil 2 dan 21.45% pada Kuartil 3

Sehingga untuk pada Kuartal 4 layak untuk mendapatkan investor baru dan akan mendapatkan untung yang lebih tinggi dibandingkan Kuartil 1 hingga Kuartil 3.

In [ ]:
# GMV (Gross Merchendise Value/Nilai barang dagangan bruto)

df_GMV = client.query ('''                                                               # Membuat variabel df_1 untuk Penjabaran 1
SELECT
      Kuartil, User, Transaksi, GMV
FROM (
      SELECT
            EXTRACT (quarter from created_at) AS Kuartil,
            COUNT (DISTINCT user_id) AS User,
            COUNT (order_id) AS Transaksi,
            SUM (retail_price) AS GMV
      FROM
            bigquery-public-data.thelook_ecommerce.order_items AS Order_Items
      JOIN
            bigquery-public-data.thelook_ecommerce.products AS Products
      ON
           Order_Items.product_id = Products.id
      WHERE
           status = 'Complete' and created_at BETWEEN "2022-01-01" AND "2022-12-31"
      GROUP BY
           Kuartil
      HAVING
           Kuartil < 4
      )
ORDER BY
      Kuartil
''').to_dataframe()

df_GMV['Persen User'] = df_GMV['User'].pct_change().mul(100)
df_GMV['Persen Transaksi'] = df_GMV['Transaksi'].pct_change().mul(100)
df_GMV['Persen GMV'] = df_GMV['GMV'].pct_change().mul(100)
df_GMV

,Kuartil,User,Transaksi,GMV,Persen User,Persen Transaksi,Persen GMV
0,1,2045,3053,189091.360189,<NA>,<NA>,NaN
1,2,2316,3375,208778.120243,13.251834,10.547003,10.411242
2,3,2844,4099,245986.030226,22.797927,21.451852,17.821748
